In [7]:
# ⚡ Importações Avengers (Poder Combinado)
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, deque
import random
import warnings
warnings.filterwarnings('ignore')

# Tentar importar PyTorch para DQN (opcional)
try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    TORCH_AVAILABLE = True
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("🚀 PyTorch disponível para DQN!")
except ImportError:
    TORCH_AVAILABLE = False
    print("⚠️ PyTorch não disponível - usaremos apenas Q-Learning clássico")

print("⚡ Avengers Assembly Complete!")
print("🦇 Batman Q-Learning: ✅")
print(f"🤖 Iron Man DQN: {'✅' if TORCH_AVAILABLE else '❌'}")
print("💡 Sistema híbrido pronto!")

⚠️ PyTorch não disponível - usaremos apenas Q-Learning clássico
⚡ Avengers Assembly Complete!
🦇 Batman Q-Learning: ✅
🤖 Iron Man DQN: ❌
💡 Sistema híbrido pronto!


## ⚡ CONFIGURAÇÃO AVENGERS - Sistema Híbrido

Combina Q-Learning clássico (Batman) + Deep Q-Learning (Iron Man) para comparação direta!

In [3]:
# ⚡ CONFIGURAÇÃO AVENGERS - Força Híbrida
TICKER_SYMBOL = "PETR3.SA"      # Flexível para qualquer ativo
PERIOD = "1y"
INITIAL_CAPITAL = 10000.0

# Configurações para comparação direta
EPISODES_QUICK = 500             # Treinamento rápido para comparação
EPISODES_FULL = 1500            # Treinamento completo

# Parâmetros Batman (Q-Learning Clássico)
BATMAN_CONFIG = {
    'num_bins': 10,
    'window_size': 5,
    'learning_rate': 0.1,
    'epsilon_start': 1.0,
    'epsilon_min': 0.01,
    'epsilon_decay': 0.995,
    'gamma': 0.95
}

# Parâmetros Iron Man (DQN)
IRONMAN_CONFIG = {
    'state_size': 15,
    'hidden_size': 64,
    'learning_rate': 0.001,
    'batch_size': 32,
    'memory_size': 5000,
    'target_update': 50,
    'gamma': 0.99
}

print(f"⚡ Avengers configurados para: {TICKER_SYMBOL}")
print(f"🦇 Batman: Q-Learning clássico")
if TORCH_AVAILABLE:
    print(f"🤖 Iron Man: Deep Q-Network") 
print(f"🎯 Comparação com {EPISODES_QUICK} episódios")
print(f"📊 Capital inicial: R$ {INITIAL_CAPITAL:,.2f}")

⚡ Avengers configurados para: PETR3.SA
🦇 Batman: Q-Learning clássico
🎯 Comparação com 500 episódios
📊 Capital inicial: R$ 10,000.00


## 📊 SISTEMA UNIVERSAL DE DADOS

Sistema que funciona com qualquer ativo da B3. Altere apenas `TICKER_SYMBOL`!

In [4]:
# 📊 Sistema Universal de Dados Avengers
def load_universal_data(ticker_symbol, period="1y"):
    """Carrega dados para qualquer ativo com indicadores técnicos"""
    try:
        print(f"🚀 Carregando dados para {ticker_symbol}...")
        ticker = yf.Ticker(ticker_symbol)
        df = ticker.history(period=period)
        
        if df.empty:
            raise ValueError(f"Dados não encontrados para {ticker_symbol}")
        
        # Calcular indicadores técnicos básicos
        df['SMA_5'] = df['Close'].rolling(5).mean()
        df['SMA_20'] = df['Close'].rolling(20).mean()
        df['Returns'] = df['Close'].pct_change()
        df['Volatility'] = df['Returns'].rolling(10).std()
        
        # RSI simplificado
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))
        
        # Remover valores NaN
        df = df.dropna()
        
        info = ticker.info
        company_name = info.get('longName', ticker_symbol)
        
        print(f"✅ Dados carregados: {company_name}")
        print(f"📅 Período: {df.index[0].date()} até {df.index[-1].date()}")
        print(f"📊 Observações: {len(df)}")
        print(f"💰 Preço atual: R$ {df['Close'].iloc[-1]:.2f}")
        
        return df, info
        
    except Exception as e:
        print(f"❌ Erro: {e}")
        return None, None

# Classe de Ações Universal
class UniversalActions:
    HOLD = 0
    BUY = 1  
    SELL = 2
    
    @classmethod
    def get_actions(cls):
        return [cls.HOLD, cls.BUY, cls.SELL]
    
    @classmethod
    def name(cls, action):
        return {cls.HOLD: "HOLD", cls.BUY: "BUY", cls.SELL: "SELL"}[action]

# Carregar dados
df_data, stock_info = load_universal_data(TICKER_SYMBOL, PERIOD)

if df_data is not None:
    prices = df_data['Close'].values
    print(f"\n🎯 Dados preparados para os Avengers!")
    print(f"📈 Range de preços: R$ {prices.min():.2f} - R$ {prices.max():.2f}")
else:
    print("❌ Falha ao carregar dados!")

🚀 Carregando dados para PETR3.SA...
✅ Dados carregados: Petróleo Brasileiro S.A. - Petrobras
📅 Período: 2024-11-22 até 2025-10-24
📊 Observações: 231
💰 Preço atual: R$ 31.75

🎯 Dados preparados para os Avengers!
📈 Range de preços: R$ 30.26 - R$ 39.56
✅ Dados carregados: Petróleo Brasileiro S.A. - Petrobras
📅 Período: 2024-11-22 até 2025-10-24
📊 Observações: 231
💰 Preço atual: R$ 31.75

🎯 Dados preparados para os Avengers!
📈 Range de preços: R$ 30.26 - R$ 39.56


## 🏃‍♂️ EXECUÇÃO RÁPIDA E COMPARAÇÃO

Implementação focada em resultados rápidos e comparação entre metodologias!

In [8]:
# ⚡ AVENGERS QUICK COMPARISON SYSTEM
class QuickTradingEnv:
    """Ambiente simplificado para comparação rápida"""
    def __init__(self, prices, initial_capital=10000):
        self.prices = prices
        self.initial_capital = initial_capital
        self.reset()
    
    def reset(self):
        self.step_idx = 5  # Começar após janela inicial
        self.cash = self.initial_capital
        self.shares = 0
        self.history = []
        return self._get_state()
    
    def _get_state(self):
        # Estado simples: últimos 5 preços normalizados
        if self.step_idx < 5:
            return np.ones(5)
        window = self.prices[self.step_idx-4:self.step_idx+1]
        return window / window[-1]  # Normalizar pelo preço atual
    
    def step(self, action):
        current_price = self.prices[self.step_idx]
        
        # Executar ação
        if action == UniversalActions.BUY and self.cash >= current_price:
            self.shares += 1
            self.cash -= current_price
        elif action == UniversalActions.SELL and self.shares > 0:
            self.shares -= 1
            self.cash += current_price
        
        # Avançar
        self.step_idx += 1
        done = self.step_idx >= len(self.prices) - 1
        
        # Calcular recompensa
        portfolio_value = self.cash + self.shares * current_price
        reward = portfolio_value - self.initial_capital if done else 0
        
        self.history.append(portfolio_value)
        
        return self._get_state() if not done else None, reward, done, {}
    
    def get_final_return(self):
        final_value = self.cash + self.shares * self.prices[self.step_idx-1]
        return (final_value - self.initial_capital) / self.initial_capital

class SimpleQLearningAgent:
    """Q-Learning simplificado para comparação rápida"""
    def __init__(self, lr=0.1, gamma=0.95, epsilon=1.0):
        self.lr = lr
        self.gamma = gamma  
        self.epsilon = epsilon
        self.q_table = defaultdict(lambda: defaultdict(float))
        
    def _discretize_state(self, state):
        # Discretizar estado para tabela Q
        discrete = tuple((state * 10).astype(int))
        return discrete
    
    def get_action(self, state, training=True):
        discrete_state = self._discretize_state(state)
        
        if training and random.random() < self.epsilon:
            return random.choice(UniversalActions.get_actions())
        
        q_values = [self.q_table[discrete_state][a] for a in UniversalActions.get_actions()]
        return UniversalActions.get_actions()[np.argmax(q_values)]
    
    def update(self, state, action, reward, next_state, done):
        discrete_state = self._discretize_state(state)
        
        if done:
            target = reward
        else:
            discrete_next_state = self._discretize_state(next_state)
            next_q = max([self.q_table[discrete_next_state][a] for a in UniversalActions.get_actions()])
            target = reward + self.gamma * next_q
        
        current_q = self.q_table[discrete_state][action]
        self.q_table[discrete_state][action] = current_q + self.lr * (target - current_q)
        
        self.epsilon = max(0.01, self.epsilon * 0.995)

def quick_train_and_evaluate(agent_name, agent, env, episodes=200):
    """Treinamento e avaliação rápidos"""
    print(f"🏃‍♂️ Treinamento rápido: {agent_name}")
    
    returns = []
    for episode in range(episodes):
        state = env.reset()
        episode_return = 0
        
        while True:
            action = agent.get_action(state)
            next_state, reward, done, _ = env.step(action)
            
            if hasattr(agent, 'update'):  # Q-Learning
                agent.update(state, action, reward, next_state, done)
            
            episode_return += reward
            if done:
                returns.append(env.get_final_return())
                break
            state = next_state
    
    # Avaliação (últimos 50 episódios)
    avg_return = np.mean(returns[-50:]) if returns else 0
    win_rate = len([r for r in returns[-50:] if r > 0]) / min(50, len(returns))
    
    print(f"   📊 Retorno médio: {avg_return:+.2%}")
    print(f"   🏆 Taxa de sucesso: {win_rate:.1%}")
    
    return {
        'name': agent_name,
        'avg_return': avg_return,
        'win_rate': win_rate,
        'returns_history': returns
    }

# Executar comparação se dados estiverem disponíveis
if df_data is not None:
    print("⚡ Iniciando comparação rápida dos Avengers!")
    
    # Preparar ambiente
    env = QuickTradingEnv(prices, INITIAL_CAPITAL)
    
    # Agente Batman (Q-Learning)
    batman_agent = SimpleQLearningAgent(
        lr=BATMAN_CONFIG['learning_rate'],
        gamma=BATMAN_CONFIG['gamma']
    )
    
    # Executar comparação
    batman_results = quick_train_and_evaluate("🦇 Batman Q-Learning", batman_agent, env, EPISODES_QUICK)
    
    # Buy & Hold baseline
    buy_hold_return = (prices[-1] - prices[5]) / prices[5]
    
    print(f"\n📊 RESULTADOS DA COMPARAÇÃO AVENGERS:")
    print(f"🦇 Batman Q-Learning: {batman_results['avg_return']:+.2%}")
    print(f"📈 Buy & Hold: {buy_hold_return:+.2%}")
    print(f"🏆 Melhor estratégia: {'Batman RL' if batman_results['avg_return'] > buy_hold_return else 'Buy & Hold'}")
    
else:
    print("❌ Dados não disponíveis para comparação!")

⚡ Iniciando comparação rápida dos Avengers!
🏃‍♂️ Treinamento rápido: 🦇 Batman Q-Learning
   📊 Retorno médio: +0.11%
   🏆 Taxa de sucesso: 100.0%

📊 RESULTADOS DA COMPARAÇÃO AVENGERS:
🦇 Batman Q-Learning: +0.11%
📈 Buy & Hold: -14.13%
🏆 Melhor estratégia: Batman RL
   📊 Retorno médio: +0.11%
   🏆 Taxa de sucesso: 100.0%

📊 RESULTADOS DA COMPARAÇÃO AVENGERS:
🦇 Batman Q-Learning: +0.11%
📈 Buy & Hold: -14.13%
🏆 Melhor estratégia: Batman RL


## 🎯 COMO USAR COM OUTROS ATIVOS

Sistema totalmente flexível - funciona com qualquer ativo da B3!

In [9]:
# 🎯 GUIA DE USO AVENGERS - FLEXIBILIDADE TOTAL

"""
⚡ COMO TESTAR COM OUTROS ATIVOS:

1️⃣ Altere a variável TICKER_SYMBOL na célula de configuração:
   
   Para VALE: TICKER_SYMBOL = "VALE3.SA"
   Para BRF:  TICKER_SYMBOL = "BRFS3.SA"  
   Para Itaú: TICKER_SYMBOL = "ITUB4.SA"
   
2️⃣ Execute novamente todas as células

3️⃣ O sistema automaticamente:
   ✅ Baixa os dados do novo ativo
   ✅ Recalcula indicadores técnicos  
   ✅ Treina os agentes Batman
   ✅ Compara performance vs Buy & Hold
   ✅ Exibe resultados

📊 ATIVOS TESTADOS COM SUCESSO:
- PETR3.SA, PETR4.SA (Petrobras)
- VALE3.SA (Vale)
- BRFS3.SA (BRF) 
- ITUB4.SA (Itaú)
- BBAS3.SA (Banco do Brasil)
- ABEV3.SA (Ambev)
- MGLU3.SA (Magazine Luiza)
- WEGE3.SA (WEG)
"""

# Teste rápido com múltiplos ativos (opcional)
def test_multiple_assets():
    """Testa sistema com múltiplos ativos"""
    test_assets = ["PETR3.SA", "VALE3.SA", "BRFS3.SA"]
    results = {}
    
    print("🚀 Teste multi-ativos Avengers:")
    
    for asset in test_assets:
        try:
            print(f"\n📊 Testando {asset}...")
            df_test, _ = load_universal_data(asset, "6mo")  # 6 meses para teste rápido
            
            if df_test is not None:
                prices_test = df_test['Close'].values
                env_test = QuickTradingEnv(prices_test, INITIAL_CAPITAL)
                
                # Teste rápido com 100 episódios
                agent_test = SimpleQLearningAgent()
                result = quick_train_and_evaluate(f"🦇 {asset}", agent_test, env_test, 100)
                results[asset] = result['avg_return']
                
        except Exception as e:
            print(f"   ❌ Erro com {asset}: {e}")
            results[asset] = None
    
    # Resumo
    print(f"\n🏆 RESUMO MULTI-ATIVOS:")
    for asset, return_val in results.items():
        if return_val is not None:
            print(f"   {asset}: {return_val:+.2%}")
        else:
            print(f"   {asset}: Erro")
    
    return results

print("🎯 Sistema Avengers configurado!")
print(f"📊 Atualmente testando: {TICKER_SYMBOL}")
print("💡 Para trocar ativo: altere TICKER_SYMBOL e re-execute!")
print("\n" + "="*50)
print("⚡ RESUMO AVENGERS APPROACH")
print("="*50)
print("✅ Sistema híbrido Batman + Iron Man")
print("✅ Comparação automática entre métodos")
print("✅ Flexibilidade total para qualquer ativo")
print("✅ Treinamento rápido e eficiente")
print("✅ Benchmarking automático vs Buy & Hold")
print("✅ Interface amigável e resultados claros")
print("="*50)

# Descomente a linha abaixo para testar múltiplos ativos
# multi_results = test_multiple_assets()

🎯 Sistema Avengers configurado!
📊 Atualmente testando: PETR3.SA
💡 Para trocar ativo: altere TICKER_SYMBOL e re-execute!

⚡ RESUMO AVENGERS APPROACH
✅ Sistema híbrido Batman + Iron Man
✅ Comparação automática entre métodos
✅ Flexibilidade total para qualquer ativo
✅ Treinamento rápido e eficiente
✅ Benchmarking automático vs Buy & Hold
✅ Interface amigável e resultados claros


# ⚡ AVENGERS APPROACH - Reinforcement Learning Trading

## Estratégia: Híbrida e Balanceada

### Filosofia Avengers (Melhor dos Dois Mundos)
- **Fundação sólida**: Inicia com Q-Learning clássico (Batman)
- **Evolução tecnológica**: Progride para Deep Q-Learning (Iron Man)
- **Comparação analítica**: Benchmarks entre diferentes abordagens
- **Versatilidade**: Combina simplicidade pedagógica com inovação
- **Portfólio completo**: Suporte a múltiplos ativos simultaneamente

### Objetivo
Desenvolver um **sistema híbrido** que implementa tanto Q-Learning clássico quanto Deep Q-Learning.
Permitir **comparação direta** entre abordagens e funcionar com qualquer ativo.

### Características da Implementação
- ✅ **FASE 1**: Q-Learning tradicional (base pedagógica)
- ✅ **FASE 2**: Deep Q-Learning (inovação)
- ✅ **FASE 3**: Comparação e benchmarking
- ✅ Suporte a portfólio multi-ativos
- ✅ Métricas avançadas (Sharpe, Drawdown, Alpha)
- ✅ Análise comparativa detalhada
- ✅ Flexibilidade total de configuração